In [36]:
## imports
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (
    StratifiedKFold,  # use X = np.zeros(n_samples) in .split
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

from img_data import Img_Obj_Dataset

# if you have indeces and want knns from large set, can make an idx array and NearestVectorCaller._call_vec_set; can just train/test split np.arange(len(ds)) to get back indeces for large sets


from btv import *

In [37]:
## Data setup

ds = Img_Obj_Dataset("data/hardhat/test", max_dim=256)
# can pass metric == "cosine", get from NearestVectorCaller.metric on large sets
knn = KNeighborsClassifier(weights="distance", n_neighbors=5) #, metric = "cosine")
# X, y = ds.whole_img_vec_set[:] # only need y for class balancing... ds.whole_img_vec_set.y

idx_train, idx_test = train_test_split(
    np.arange(len(ds.whole_img_vec_set)), test_size=0.2
)

Detemining all classes...: 100%|██████████████████████████████| 1766/1766 [00:00<00:00, 2149.52it/s]


In [38]:
knn.fit(ds.whole_img_vec_set.X[idx_train], ds.whole_img_vec_set.y[idx_train])
# knn.predict(X_test)
# ds.whole_img_vec_set.X[idx_test]

KNeighborsClassifier(weights='distance')

In [39]:
accuracy_score(
    ds.whole_img_vec_set.y[idx_test], knn.predict(ds.whole_img_vec_set.X[idx_test])
)

0.748587570621469

In [9]:
# only include p if y==1 or, reeally, want p(right answer) so take elemnt of predicted prob indicated by binary y.
def prediction_entropy(y_true, y_predicted): # multilable
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    A = np.zeros_like(y_true)
    with np.errstate(divide="ignore"):
        for i, (pred, true) in enumerate(zip(y_predicted, y_true)):
            for j, (p, t) in enumerate(zip(pred, true)):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                A[i, j] = I
    return np.mean(A, axis=0)


# probably actually want to do this at the prediction level since that is the big/slow bit.
def prediction_entropy_lg(y_true, y_predicted): # sinlge label
    # replace log(0) with n instead of -Inf
    if isinstance(y_predicted, list):
        y_predicted = np.stack(y_predicted, axis=1)
    with np.errstate(divide="ignore"):
        for pred, true in zip(y_predicted, y_true):
            for p, t in zip(pred, true):
                I = -np.log(p[t])
                if np.isinf(I):
                    I = y_true.size
                yield I

In [40]:
A = knn.predict(ds.whole_img_vec_set.X[idx_test])
P = knn.predict_proba(
    ds.whole_img_vec_set.X[idx_test]
)  # predicts the probability of each class. For whole image hardhats it is a list of 3 (p(no), P(yes)) arrays

In [41]:
prediction_entropy(y_predicted, y_true)

AttributeError: 'list' object has no attribute 'shape'

In [17]:
kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
# can just add all the labels to make it multiclass for splitting
entropies = []
running_train = []
for k, (train_idx, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
    # knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    # y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[test_idx])
    # y_true = ds.whole_img_vec_set.y[test_idx]
    # H = prediction_entropy(y_true, y_predicted)
    # entropies.append(H)
    # print(f"Fold : {k+1}, " f"Fold entropy : {H}")

In [20]:
np.mean(entropies, axis=0), np.std(entropies, axis=0), np.std(
    entropies, axis=0
) / np.mean(entropies, axis=0)

(array([75.04532578, 57.61076487, 16.80113314]),
 array([38.20580196, 28.99206553,  8.40056667]),
 array([0.50910302, 0.50324042, 0.50000001]))

In [188]:
knn.predict(ds.whole_img_vec_set.X[test_idx])

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [100]:
np.sum(ds.whole_img_vec_set.y, axis=0)

array([ 339, 1604,   45])

In [43]:
## with straigh vectorization of whole images, the knn accuracy goes down as more data is added. Therefore that vecotrization is shit. Maybe we try tabular data?

kfold_idx_gen = StratifiedKFold(n_splits=5).split(
    np.zeros(len(ds.whole_img_vec_set)), np.sum(ds.whole_img_vec_set.y, axis=1)
)
train_idx = np.array([], int)
running_train = []
for k, (_, test_idx) in enumerate(kfold_idx_gen):
    running_train.append(test_idx)
running_test = running_train.pop()
for k, old_test_idx in enumerate(running_train):
    train_idx = np.append(train_idx,old_test_idx)
    knn.fit(ds.whole_img_vec_set.X[train_idx], ds.whole_img_vec_set.y[train_idx])
    y_predicted = knn.predict_proba(ds.whole_img_vec_set.X[running_test])
    y_true = ds.whole_img_vec_set.y[running_test]
    H = prediction_entropy(y_true, y_predicted)
    entropies.append(H)
    print(f"Fold : {k+1}, " f"Test set entropy : {np.mean(H)}", f"Train samples : {len(train_idx)}", f"Accuracy : {accuracy_score(y_true,knn.predict(ds.whole_img_vec_set.X[test_idx]))}")

Fold : 1, Test set entropy : 45.036827195467424 Train samples : 354 Accuracy : 0.7705382436260623
Fold : 2, Test set entropy : 45.040604343720496 Train samples : 707 Accuracy : 0.7648725212464589
Fold : 3, Test set entropy : 47.038715769593956 Train samples : 1060 Accuracy : 0.7620396600566572
Fold : 4, Test set entropy : 52.03305004721435 Train samples : 1413 Accuracy : 0.7592067988668555


In [42]:
hs, ss = add_stratified_folds_test(ds.whole_img_vec_set.X,ds.whole_img_vec_set.y,knn)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

In [21]:
running_train

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177